# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 10:52AM,254650,15,9013818,"Brookfield Pharmaceuticals, LLC",Released
1,Jan 10 2023 10:52AM,254650,15,9013819,"Brookfield Pharmaceuticals, LLC",Released
2,Jan 10 2023 10:52AM,254650,15,9013820,"Brookfield Pharmaceuticals, LLC",Released
3,Jan 10 2023 10:52AM,254650,15,9013821,"Brookfield Pharmaceuticals, LLC",Released
4,Jan 10 2023 10:52AM,254650,15,9013822,"Brookfield Pharmaceuticals, LLC",Released
5,Jan 10 2023 10:52AM,254650,15,9013823,"Brookfield Pharmaceuticals, LLC",Released
6,Jan 10 2023 10:52AM,254650,15,9013824,"Brookfield Pharmaceuticals, LLC",Released
7,Jan 10 2023 10:52AM,254650,15,9013825,"Brookfield Pharmaceuticals, LLC",Released
8,Jan 10 2023 10:52AM,254650,15,9013826,"Brookfield Pharmaceuticals, LLC",Released
9,Jan 10 2023 10:52AM,254650,15,9013827,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,254646,Released,7
39,254647,Released,3
40,254648,Released,1
41,254649,Released,16
42,254650,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254646,NaN,NaN,7.0
254647,NaN,NaN,3.0
254648,NaN,NaN,1.0
254649,NaN,NaN,16.0
254650,NaN,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254563,37.0,3.0,0.0
254566,0.0,1.0,35.0
254567,0.0,0.0,10.0
254597,0.0,1.0,0.0
254598,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254563,37,3,0
254566,0,1,35
254567,0,0,10
254597,0,1,0
254598,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254563,37,3,0
1,254566,0,1,35
2,254567,0,0,10
3,254597,0,1,0
4,254598,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254563,37,3,
1,254566,,1,35
2,254567,,,10
3,254597,,1,
4,254598,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC"
18,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc."
34,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc."
35,Jan 10 2023 10:46AM,254646,10,Eywa Pharma Inc.
42,Jan 10 2023 10:45AM,254645,10,Bio-PRF
50,Jan 10 2023 10:43AM,254647,19,"Methapharm, Inc."
53,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc."
63,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc."
78,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation
137,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",,,18
1,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",,,16
2,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",,,1
3,Jan 10 2023 10:46AM,254646,10,Eywa Pharma Inc.,,,7
4,Jan 10 2023 10:45AM,254645,10,Bio-PRF,,,8
5,Jan 10 2023 10:43AM,254647,19,"Methapharm, Inc.",,,3
6,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",,7,3
7,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",,,15
8,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,,,59
9,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",,1,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",18,,
1,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",16,,
2,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",1,,
3,Jan 10 2023 10:46AM,254646,10,Eywa Pharma Inc.,7,,
4,Jan 10 2023 10:45AM,254645,10,Bio-PRF,8,,
5,Jan 10 2023 10:43AM,254647,19,"Methapharm, Inc.",3,,
6,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",3,7,
7,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",15,,
8,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,59,,
9,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",3,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",18,,
1,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",16,,
2,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",1,,
3,Jan 10 2023 10:46AM,254646,10,Eywa Pharma Inc.,7,,
4,Jan 10 2023 10:45AM,254645,10,Bio-PRF,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",18.0,NaN,NaN
1,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",16.0,NaN,NaN
2,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Jan 10 2023 10:46AM,254646,10,Eywa Pharma Inc.,7.0,NaN,NaN
4,Jan 10 2023 10:45AM,254645,10,Bio-PRF,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",18.0,0.0,0.0
1,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",16.0,0.0,0.0
2,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Jan 10 2023 10:46AM,254646,10,Eywa Pharma Inc.,7.0,0.0,0.0
4,Jan 10 2023 10:45AM,254645,10,Bio-PRF,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2546389,106.0,3.0,5.0
15,2546099,94.0,23.0,37.0
19,1527706,55.0,3.0,0.0
20,509268,4.0,7.0,0.0
21,763794,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2546389,106.0,3.0,5.0
1,15,2546099,94.0,23.0,37.0
2,19,1527706,55.0,3.0,0.0
3,20,509268,4.0,7.0,0.0
4,21,763794,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,106.0,3.0,5.0
1,15,94.0,23.0,37.0
2,19,55.0,3.0,0.0
3,20,4.0,7.0,0.0
4,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,106.0
1,15,Released,94.0
2,19,Released,55.0
3,20,Released,4.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,5.0,37.0,0.0,0.0,0.0
Executing,3.0,23.0,3.0,7.0,1.0
Released,106.0,94.0,55.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,5.0,37.0,0.0,0.0,0.0
1,Executing,3.0,23.0,3.0,7.0,1.0
2,Released,106.0,94.0,55.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,5.0,37.0,0.0,0.0,0.0
1,Executing,3.0,23.0,3.0,7.0,1.0
2,Released,106.0,94.0,55.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()